In [238]:
import pandas as pd

path = r"C:/Users/海棠微雨/Desktop/cumcm2021c/C/附件1 近5年402家供应商的相关数据.xlsx"
data_class = pd.read_excel(path, sheet_name='供应商的供货量（m³）')

data = data_class.iloc[:, -1].values

In [ ]:
# 假设我们有50个数据
target_sum = 6568000  # 目标值

In [ ]:
#以下是一个基于 `DEAP` 库的标准遗传算法框架，包括详细注释，示例数据，和对各参数的解释。这个模型可以应用于优化问题，展示如何使用 `DEAP` 进行遗传算法操作。

from deap import base, creator, tools, algorithms
import random
import matplotlib.pyplot as plt
import numpy as np

# 定义适应度类，目标是最小化
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# 定义个体类，每个个体由三个变量 x, y, z 组成
creator.create("Individual", list, fitness=creator.FitnessMin)


# 初始化个体，x, y, z 的取值范围设为 [-10, 10]
def initIndividual(icls):
    return icls([random.randint(0, 1) for _ in range(402)])

# 初始化种群
def initPopulation(pcls, ind_init, size):
    return pcls(ind_init() for _ in range(size))

# 目标函数
def evalFunction(individual):
    selected_data = [data[i] for i in range(len(individual)) if individual[i] == 1]
    total_sum = sum(selected_data)
    num_selected = sum(individual)
    return (num_selected,)

toolbox = base.Toolbox()

# 注册初始化函数
toolbox.register("individual", initIndividual, creator.Individual)
toolbox.register("population", initPopulation, list, toolbox.individual)

# 注册评估函数、交叉函数、变异函数、选择函数
# toolbox.register("attr_float", random.uniform, -5, 5)  # 示例的初始化函数
toolbox.register("evaluate", evalFunction)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    # 生成初始种群，种群大小为50
    population = toolbox.population(size=50)

    # 设置遗传算法的参数
    cxpb, mutpb, ngen = 0.7, 0.3, 40  # 交叉概率、变异概率、迭代次数

    # 统计信息
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("avg", lambda x: sum(x) / len(x))
    stats.register("min", min)
    stats.register("max", max)
    stats.register("std", lambda x: np.std(x))  # 添加标准差统计信息

  # 记录日志
    logbook = tools.Logbook()
    logbook.header = ["gen", "nevals"] + stats.fields


    # 执行遗传算法
    population, logbook = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, 
                                              stats=stats, halloffame=None, verbose=True)
    
    # 输出结果
    best_individual = tools.selBest(population, 1)[0]
    print(f"Best individual: {best_individual}, with fitness: {best_individual.fitness.values[0]}")

# 找到最小的评估次数
    min_nevals = min(logbook.select("nevals"))
    print(f"Minimum evaluations after convergence: {min_nevals}")

    # 绘制统计信息
    generations = logbook.select("gen")
    avg_fitness = logbook.select("avg")
    min_fitness = logbook.select("min")
    max_fitness = logbook.select("max")
    std_fitness = logbook.select("std")
    y = np.full(len(generations), -6816000)


    plt.figure(figsize=(12, 8))
    plt.plot(generations, avg_fitness, label="Average Fitness")
    plt.plot(generations, min_fitness, label="Minimum Fitness")
    plt.plot(generations, max_fitness, label="Maximum Fitness")
    plt.plot(generations, std_fitness, label="Fitness Std Dev")
    plt.plot(generations, y)
    plt.xlabel("Generation")
    plt.ylabel("Fitness")
    plt.legend()
    plt.title("Fitness Progression")
    plt.xlim(0, 40)
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
import random
from deap import base, creator, tools, algorithms

# 假设我们有50个数据
target_sum = 6568000  # 目标值

# Step 1: 创建适应度函数和个体
creator.create("FitnessMin", base.Fitness, weights=(-1.0, 1.0))  # 最小化数据数量，最大化数据之和
creator.create("Individual", list, fitness=creator.FitnessMin)

# Step 2: 初始化种群
def init_individual():
    return creator.Individual([random.randint(0, 1) for _ in range(402)])

def init_population(n=402):
    return [init_individual() for _ in range(n)]

# Step 3: 定义适应度函数
def evaluate(individual):
    selected_data = [data[i] for i in range(len(individual)) if individual[i] == 1]
    total_sum = sum(selected_data)
    num_selected = sum(individual)
    if total_sum >= target_sum:
        return num_selected, total_sum
    else:
        return 50, 0  # 如果不满足条件，返回一个较差的适应度

# Step 4: 定义遗传算法操作
toolbox = base.Toolbox()
toolbox.register("mate", tools.cxTwoPoint)  # 两点交叉
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # 翻转突变
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Step 5: 遗传算法主程序
def main():
    population = init_population()
    
    for individual in population:
        individual.fitness.values = toolbox.evaluate(individual)
    
    NGEN = 40
    CXPB, MUTPB = 0.7, 0.3

    for gen in range(NGEN):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        
        for individual in offspring:
            if not individual.fitness.valid:
                individual.fitness.values = toolbox.evaluate(individual)
        
        population[:] = offspring
        fits = [ind.fitness.values[0] for ind in population]
        min_fitness = min(fits)
        best_individual = population[fits.index(min_fitness)]
        print(f"Generation {gen}: Min fitness = {min_fitness}, Best individual = {best_individual}")
    
    final_best_individual = min(population, key=lambda ind: ind.fitness.values[0])
    selected_data = [data[i] for i in range(len(final_best_individual)) if final_best_individual[i] == 1]
    print(f"Final best individual: {final_best_individual}, Selected data: {selected_data}, Sum: {sum(selected_data)}, Number of items: {sum(final_best_individual)}")
    
    return final_best_individual

if __name__ == "__main__":
    final_best_individual = main()


最优的一份

In [ ]:
import pandas as pd
from deap import base, creator, tools, algorithms
import random
import matplotlib.pyplot as plt

# 读取数据
file_path = 'D:/桌面/前50家商家供货量.xlsx'
data = pd.read_excel(file_path)
supplies = list(data['5年总产值'])

# 定义适应度函数
def evalSupply(individual, supplies):
     total_sum = sum(ind * supply for ind, supply in zip(individual, supplies))
    
     # 如果总和小于600万，返回一个高惩罚值以降低该个体的适应度
     if total_sum < 6200000:
        return 1000000 + (6400000 - total_sum),
    
     # 返回选择的数目的个数作为适应度值（越少越好）
     return sum(individual),

# 定义遗传算法
def genetic_algorithm(supplies):
    # 初始化DEAP的基本设置
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    toolbox = base.Toolbox()
    
    # 二进制编码
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, len(supplies))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # 注册适应度函数
    toolbox.register("evaluate", evalSupply, supplies=supplies)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    # 初始化种群
    population = toolbox.population(n=300)
    
    # 使用遗传算法进行优化
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, verbose=True)
    
    # 返回最优解
    best_individual = tools.selBest(population, k=1)[0]
    return best_individual, sum(best_individual[i] * supplies[i] for i in range(len(supplies)))

# 运行遗传算法
best_individual, best_sum = genetic_algorithm(supplies)

# 打印结果
print("最优解：", best_individual)
print("最优解的总供货量：", best_sum)
print("使用的商家数量：", sum(best_individual))
